# Introducción al análisis bioinformático de secuencias RADseq
### Ejemplo carancho austral
##### Santiago Ceballos, Ulises Balza y Nicolas Lois
##### Ushuaia, febrero de 2023

En este ejercicio vamos a repetir procesos que ya trabajaron en el curso, pero a traves de un _script_. De esta manera, vamos a guardar registro reproducible de cada paso y podremos correr todo el flujo de trabajo con control de lo que va pasando.
Para simplificar el ejercicio, partimos de secuencias ya demultiplexadas y vamos a dedicarnos al alineamiento de las muestras y el análisis descriptivo de la estructura de los datos.

------------------------------------------

Al final de este ejercicio ustedes podrán:
* Repasar los conceptos, funciones y programas que aprendimos a usar a lo largo de todo el curso
* Llevarse un script funcionando que puedan adaptar a sus casos
* Tener tiempo de sacarse dudas, pensar variantes para sus propios trabajos, y discutir con el resto de la clase próximos pasos a seguir





### ¿Qué es este texto tan lindo que estoy viendo?

Esto es un código escrito en Markdown, un lenguaje para crear texto con formato a partir de texto plano (sin formato).

En Markdown se puede poner cualquier cosa, por ejemplo un tuit de la primera semana del curso:

<blockquote class="twitter-tweet" data-lang="en" data-theme="dark"><p lang="es" dir="ltr">Primera camada de estudiantes que se van con sus bibliotecas RAD-seq 🧬 para <a href="https://twitter.com/Exactas_UBA?ref_src=twsrc%5Etfw">@Exactas_UBA</a> 🦗<a href="https://twitter.com/unmdp?ref_src=twsrc%5Etfw">@unmdp</a> 🦀🐟🦡 <a href="https://twitter.com/unc_cordoba?ref_src=twsrc%5Etfw">@unc_cordoba</a> 🐀 y CIT-NOBA 🐈<br>La semana que viene bioinformática 💻 <a href="https://twitter.com/hashtag/RADUSH23?src=hash&amp;ref_src=twsrc%5Etfw">#RADUSH23</a> en <a href="https://twitter.com/UNTDF?ref_src=twsrc%5Etfw">@UNTDF</a> <a href="https://t.co/Cs0ES9K1PM">pic.twitter.com/Cs0ES9K1PM</a></p>&mdash; Ulises Balza 🇦🇷 (@UlisesBalza) <a href="https://twitter.com/UlisesBalza/status/1623736774546624512?ref_src=twsrc%5Etfw">February 9, 2023</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Pero esto no es magia! Si hacés doble click sobre esta celda vas a ver que es todo texto plano. Cuando veas el código, podés ejecutar la celda (Ctrl+Enter) para volver a ver el texto formateado. 

Jupyternotebook (extensión .ipynb) es un tipo de archivo que puede compilar texto (markdown), lenguajes de programación y, más importante, resultados (output) de códigos. **Todo** va a aparecer acá. El resultado de una celda de Markdown es el texto con formato, pero el resultado de las celdas de UNIX van a ser las cosas que queremos y que estuvimos viendo.

Cada celda puede ser solo de un tipo (por ejemplo, Markdown o UNIX), pero las celdas se pueden ordenar como uno quiera para generar un documento que nos sirva, en este caso para tener registro de lo que vamos haciendo. Vamos al ejemplo y lo seguimos viendo, no desesperéis.

#### Caso de estudio
El carancho austral (*Phalcoboenus australis*) una especie endémica de islas del sur de Argentina y Chile. Pueden encontrar algunas referencias sobre la especie y el trabajo que hacemos en CADIC [acá](https://www.researchgate.net/project/Ecology-and-conservation-of-striated-caracara-Phalcoboenus-australis-in-Tierra-del-Fuego-Argentina). 
Las muestras de este trabajo fueron colectadas por nosotros y en colaboración con [Katie Harrington](https://www.johnnyrookproject.com/team-1) (Messerli Research Institute, Viena). 

Las muestras provienen de cuatro poblaciones de la especie: tres islas en Malvinas (Argentina), cuyos códigos son *bleaker*, *new_island* y *saunders* (correspondientes a nuestras islas María, Goicoechea y Trinidad respectivamente) y una en el archipiélago fueguino (Isla de los Estados, cuyo código es *idle*). 

![image](./images/figura_carancho.png)

**Figura 1** Distribución global del carancho austral y localidades de donde provienen las muestras de este trabajo

Como grupo externo, usamos una muestra de chimango (*Phalcoboenus chimango*, género *Milvago* en la figura) proveniente de la Provincia de Buenos Aires y cuyo código es *chim_02* (porque *chim_01* salió horrible, muy pocas lecturas, malísimo). El chimango es el grupo hermano de las cuatro especies de caranchos andinos a las que pertecene el carancho austral y la especie más cercana a la que tuvimos acceso en ese momento.

![image](./images/Figuras_fuchs.png)

**Figura 2** Filogenia de caracaras utilizando marcadores mitocondriales y nucleares ([Fuchs et al., 2012](https://onlinelibrary.wiley.com/doi/pdf/10.1111/j.1474-919X.2012.01222.x))


Las secuencias serán alineadas con el [genoma de referencia](https://www.ncbi.nlm.nih.gov/assembly/GCF_000337955.1/) del halcón peregrino (*Falco peregrinus*, una especie de la misma familia, ver árbol arriba). Actualmente existe un proyecto para generar genomas de referencias de todas las rapaces del mundo, pero de momento esta es la especie más cercanamente emparentada de la que disponemos un genoma completo.

#### 1. Navegar el cluster desde un notebook
Empecemos por chequear qué muestras tenemos y que estén en la carpeta adecuada.

Para eso podemos usar los tres comandos que ya conocen: `pwd`, `ls` y `cd`

¡Ojo! Al contrario de la consola, cuando corremos líneas sucesivas en una celda, se imprimen todo los resultados de corrido y eso puede resultar difícil de entender. Lo recomendable es que le pidamos especificamente que imprima unos pocos elementos y en todo caso usar celdas diferentes para cada cosa. 

_Hint_: El acceso directo para comentar y descomentar líneas es _ctrl_ + _/_ (teclado LatAm _ctrl_ + _shift_  7)

In [1]:
%%bash
pwd
# ls
# ls samples
# cd samples

/home/cursorad2023/carancho


**ATENCIÓN** Si cambian el directorio de trabajo (e.g. van a la carpeta samples con el comando `cd samples`), el código se va a ejecutar desde ese lugar.

Si usan SIEMPRE paths absolutos no hay problema, pero es recomendable trabajar desde la raíz

El argumento `%%bash` al comienzo de la celda indica que sea leído con el intérprete _bash_, el que usa la consola de Linux que estuvieron trabajando. ¿Qué pasa si no ponemos ese argumento? Podemos probar lo que querramos porque la gracia de usar un script es que queda registro de todo. Y relax, no pueden romper nada, tenemos un backup de todo esto alojado en la nube. Así que tómense el tiempo de editar, agregar celdas de código o texto, comentar todo lo que necesiten para llevarse algo que les sirva a ustedes. **Díganme que esto es mejor que un word por favor**.

#### 2. Alineamiento de las lecturas
Primero, vamos a hacer un archivo que clasifique mis muestras en poblaciones - lo vamos a llamar `popmap_all.txt`

Generamos el archivo de la lista de muestras, y a continuación le agregamos un "1" en la columna de al lado

In [2]:
%%bash 
cd samples # cambio de directorio
ls -1 *.fq.gz| sed -e 's/\.fq.gz$//' > samples_list_all.txt # acá genero la lista de muestras
awk 'BEGIN { FS = OFS = "\t" } { $(NF+1) = 1; print $0 }' samples_list_all.txt > popmap_all.txt
cat popmap_all.txt
cd .. # vuelvo al directorio original

bleaker_03	1
bleaker_06	1
bleaker_10	1
chim_02	1
idle_a24	1
idle_naca	1
idle_nacp	1
new_island_01	1
new_island_03	1
new_island_10	1
saunders_07	1
saunders_09	1
saunders_10	1


Bien, tenemos una receta para hacer las cosas. Para generar una lista de nombres de muestras hicimos un código bastante complejo y desprolijo.

Entramos a una carpeta, operamos dentro de ella (siempre con paths relativos) y luego volvimos a la carpeta de origen. ¿Se puede mejorar? Seguro, pero no necesitamos empezar de cero cada vez que entremos a este código, lo podemos ir mejorando con el tiempo y ésta no tiene por qué ser la versión final. Si bien la prolijiad es relevante, el mejor código es el que anda.

![image](./images/meme_pirates.png)

### 3. Indexado del genoma de referencia
A partir de este punto tendrán que tomar nota con los comentarios como mostramos en los puntos anteriores y correr los pasos que hicimos antes para tener información de este nuevo set de datos. Aprovechen para generar un producto que se puedan llevar y les sirva para el futuro

In [7]:
%%bash

### 4. Loop de alineamiento

### 5. SNP calling

### 6. Populations